In [1]:
import pandas as pd
import numpy as np

# APIs for accessing data
# NYC Open Data API
from sodapy import Socrata
import requests

# Set up client to access Socrata
# Example non-authenticated client
client = Socrata('data.cityofnewyork.us', None)

# 2014 - 2015 DOE High School Directory
results = client.get_all('n3p6-zve2')
# Convert to pandas DataFrame
hs_info = pd.DataFrame.from_records(results)

hs_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 69 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   dbn                               435 non-null    object
 1   school_name                       435 non-null    object
 2   boro                              435 non-null    object
 3   building_code                     435 non-null    object
 4   phone_number                      435 non-null    object
 5   fax_number                        435 non-null    object
 6   grade_span_min                    432 non-null    object
 7   grade_span_max                    435 non-null    object
 8   bus                               435 non-null    object
 9   subway                            435 non-null    object
 10  primary_address_line_1            435 non-null    object
 11  city                              435 non-null    object
 12  state_code            

In [2]:
from censusgeocode import CensusGeocode
import re
cg = CensusGeocode(benchmark='Public_AR_Current', vintage='Census2010_Current')


def return_tract(df):
    # use the census lookup to find the tract
    result = cg.address(df['primary_address_line_1'],
                        city=df['city'], state=df['state_code'], zip=df['zip'])
    # save the census tract when looking up the address
    try:
        tract_lookup = result[0]['geographies']['Census Tracts'][0]['TRACT']
        return tract_lookup
    except:
        print('\nFlag: ' + df['dbn'])
        print(df['primary_address_line_1'])
        print(df['city'] + ', ' + df['state_code'])
        print(df['zip'])
        print(df['census_tract'])
        return ''

In [3]:
# temp = hs_info.loc[hs_info['dbn'] == '29Q498']
# result = cg.address(temp.iloc[0]['primary_address_line_1'],
#                     city=temp.iloc[0]['city'], state=temp.iloc[0,]['state_code'], zip=temp.iloc[0]['zip'])

In [4]:
# temp[['school_name', 'primary_address_line_1', 'city', 'state_code', 'zip', 'census_tract']]

In [5]:
%%time
# temp = hs_info.iloc[10:12].copy()
temp = hs_info.copy()
temp['tract_lookup'] = temp.apply(return_tract, axis=1)


Flag: 29Q498
207- 01 116Th Avenue
Cambria Heights, NY
11411
53401

Flag: 02M551
550 Wheeler Avenue
Manhattan, NY
10004
nan

Flag: 25Q281
46-21 Colden Street
Flushing, NY
11355
845

Flag: 30Q301
1-50 51St Avenue
Long Island City, NY
11101
1

Flag: 27Q323
320 Beach 104Th Street
Rockaway Park, NY
11694
938

Flag: 27Q314
121-10 Rockaway Boulevard
Queens, NY
11420
840

Flag: 13K527
283 Adams Street
Brooklyn, NY
11201
11

Flag: 25Q525
149-11 Melbourne Avenue
Flushing, NY
11367
809

Flag: 12X248
1180 Rev. J.A. Polite Ave.
Bronx, NY
10459
131

Flag: 29Q492
207- 01 116Th Avenue
Cambria Heights, NY
11411
53401

Flag: 26Q315
230-17 Hillside Avenue
Queens Village, NY
11427
1301

Flag: 02M282
411 Peal Street
New York, NY
10038
nan

Flag: 26Q430
58-20 Utopia Parkway
Fresh Meadows, NY
11365
1417

Flag: 12X278
1180 Rev. J.A. Polite Ave.
Bronx, NY
10459
131

Flag: 02M489
100 Trinity Place
New York, NY
10006
13

Flag: 13K419
49 Flatbush Avenue Extension
Brooklyn, NY
11201
15

Flag: 28Q686
91-30 Metropo

In [6]:
# create a guess of the six digit census tract. This can be used to check that the 
# tract lookup is correct, and find a census tract for any schools that we couldn't lookup
def six_digit_tract(df):
    # added as conditions later; these were formatted incorrectly using the code below
    # found after comparing the guessed tract names to the looked up tract names
    if (
        (df['dbn'] == '01M292')
        | (df['dbn'] == '01M448')
        | (df['dbn'] == '01M696')
        | (df['dbn'] == '01M539')
        | (df['dbn'] == '01M509')
    ):
        return df['census_tract'].zfill(6)
    else:
        if isinstance(df['census_tract'], str):
            if len(df['census_tract']) == 6:
                return df['census_tract']
            #if 4 digist return, assume that's the 4-digit basic code and pad with 00
            elif len(df['census_tract']) <= 4:
                return df['census_tract'].zfill(4).ljust(6, '0')
            elif len(df['census_tract']) == 5:
                return df['census_tract'].zfill(6)
        # manual edits for the few float instances of census_tract
        elif isinstance(df['census_tract'], float):
            if (df['dbn'] == '02M551') | (df['dbn'] == '02M282'):
                return ''
            # manually looked up tract for this school
            elif df['dbn'] == '06M552':
                return '025100'

temp['tract_guess'] = temp.apply(six_digit_tract, axis=1)

In [7]:
# here are the cases where the census tract listed is different than what you might guess
temp[['dbn', 'school_name', 'census_tract', 'tract_lookup', 'tract_guess']].loc[(temp['tract_lookup'] != temp['tract_guess']) & (temp['tract_lookup']!='')]

,dbn,school_name,census_tract,tract_lookup,tract_guess
48,19K404,Academy for Young Writers,1070,107800,107000
63,07X500,Hostos-Lincoln Academy of Science,73,007500,007300
70,32K168,The Brooklyn School for Math and Research,435,043900,043500
131,24Q550,High School for Arts and Business,43701,045500,043701
133,32K403,Academy for Environmental Leadership,435,043900,043500
160,13K674,"City Polytechnic High School of Engineering, A...",13,001500,001300
187,13K605,George Westinghouse Career and Technical Educa...,13,001500,001300
189,32K552,Academy of Urban Planning,435,043900,043500
190,19K422,Spring Creek Community School,1070,107800,107000
198,10X237,"The Marie Curie School for Medicine, Nursing, ...",273,027700,027300


In [8]:
# here are the ones that are missing when looking up the census tract
temp[[
    'dbn', 'school_name',
    'primary_address_line_1', 'city', 'state_code', 'zip',
    'census_tract', 'tract_lookup', 'tract_guess']].loc[temp['tract_lookup']=='']

,dbn,school_name,primary_address_line_1,city,state_code,zip,census_tract,tract_lookup,tract_guess
11,29Q498,Humanities & Arts Magnet High School,207- 01 116Th Avenue,Cambria Heights,NY,11411,53401,,053401
22,02M551,Urban Assembly New York Harbor School,550 Wheeler Avenue,Manhattan,NY,10004,NaN,,
23,25Q281,East-West School of International Studies,46-21 Colden Street,Flushing,NY,11355,845,,084500
36,30Q301,Academy for Careers in Television and Film,1-50 51St Avenue,Long Island City,NY,11101,1,,000100
87,27Q323,Scholars' Academy,320 Beach 104Th Street,Rockaway Park,NY,11694,938,,093800
94,27Q314,Epic High School – South,121-10 Rockaway Boulevard,Queens,NY,11420,840,,084000
105,13K527,Urban Assembly Institute of Math and Science f...,283 Adams Street,Brooklyn,NY,11201,11,,001100
108,25Q525,Townsend Harris High School,149-11 Melbourne Avenue,Flushing,NY,11367,809,,080900
110,12X248,"Metropolitan High School, The",1180 Rev. J.A. Polite Ave.,Bronx,NY,10459,131,,013100
121,29Q492,"Mathematics, Science Research and Technology M...",207- 01 116Th Avenue,Cambria Heights,NY,11411,53401,,053401


In [9]:
# create final version of tract var; defaulting currently to tract guess
def check_tract(df):
#     if bool(re.search(df['census_tract'], df['tract_lookup'])):
    if isinstance(df['census_tract'], str):
        if df['census_tract'] in df['tract_lookup']:
            return df['tract_lookup']
        else:
            return df['tract_guess']
    return df['tract_guess']

In [10]:
temp['tract'] = temp.apply(check_tract, axis=1)

In [11]:
temp[['dbn', 'tract', 'tract_lookup', 'tract_guess', 'boro']].to_csv('dbn_tract_lookup.csv', index=False)

In [12]:
temp[['dbn', 'census_tract', 'tract', 'tract_lookup', 'tract_guess']]

,dbn,census_tract,tract,tract_lookup,tract_guess
0,27Q260,100802,100802,100802,100802
1,21K559,306,030600,030600,030600
2,16K393,291,029100,029100,029100
3,08X305,16,001600,001600,001600
4,03M485,151,015100,015100,015100
...,...,...,...,...,...
430,02M300,135,013500,013500,013500
431,09X412,225,022500,022500,022500
432,32K549,435,043500,043900,043500
433,02M407,48,004800,004800,004800
